In [12]:
# import 
import pickle
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,roc_auc_score,roc_curve
import warnings
warnings.filterwarnings('ignore')

!pwd
!ls -a ./raw

/Users/hobee/PythonScript/CCF-BDCI-2022-FXFZ
.                    dataA.csv            dataTrain.csv
..                   dataNoLabel.csv      submit_example_A.csv


In [24]:
# load the data
semi_data = pd.read_csv("./cache/semi_label_data.csv")
raw_data = pd.read_csv("./cache/label_data.csv")
label_data = pd.read_csv("./cache/label.csv")

data = semi_data.copy()

In [9]:
# load the model
model_file_path = "./models/stack_model.pkl"
model = pickle.load(open(model_file_path, "rb"))

In [13]:
def train_model(x,y, model=None):
    x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.2,random_state = 33)
    ### 训练模型
    if model == None:
        model = xgb.XGBClassifier(learning_rate=0.1,
                      n_estimators=100,            # 树的个数--100棵树建立xgboost
                      max_depth=6,                 # 树的深度
                      min_child_weight = 1,        # 叶子节点最小权重
                      gamma=0.,                    # 惩罚项中叶子结点个数前的参数
                      subsample=0.8,               # 随机选择80%样本建立决策树
                      colsample_btree=0.8,         # 随机选择80%特征建立决策树
                      objective='binary:logistic', # 指定损失函数
                      scale_pos_weight=1,          # 解决样本个数不平衡的问题
                      random_state=27              # 随机数
                      )

    # 拟合
    # model.fit(x_train, y_train, eval_set = [(x_test,y_test)], eval_metric = "auc", early_stopping_rounds = 10,verbose = True)
    model.fit(x_train, y_train)
    
    return model,x_train,x_test,y_train,y_test

In [25]:
# predict
pred_prob_A = model.predict_proba(data)
data[pred_prob_A[:,0] > 0.9]
data[pred_prob_A[:,1] > 0.9]
data[pred_prob_A[:,1] > 0.9]
label_data.append(pd.DataFrame(""))

TypeError: cannot concatenate object of type '<class 'int'>'; only Series and DataFrame objs are valid

In [ ]:
# 思路 循环训练 每次将label为概率大于90%的加入训练集重新训练 直到结果无法提升
last_auc = 0
epoch = 1
while True:
    pb = model.predict_proba(data)
    pos_item = data[pred_prob_A[:,0] > 0.9]
    neg_item = data[pred_prob_A[:,1] > 0.9]
    raw_data.append(pos_item,ignore_index=True)
    raw_data.append(neg_item,ignore_index=True)
    label_data.append(
        pd.DataFrame({"label": [1 for i in range(pos_item.shape[0])]}),
        ignore_index=True
    )
    label_data.append(
        pd.DataFrame({"label": [1 for i in range(neg_item.shape[0])]}),
        ignore_index=True
    )
    model,x_train,x_test,y_train,y_test = train_model(raw_data,label_data,model)
    if epoch != 1:
        last_auc = auc
    auc = roc_auc_score(y_test, model.predict_proba(x_test)[:,1])
    delta = auc - last_auc
    print(f"epoch:{epoch},auc:{auc},last_auc:{last_auc},dalta:{auc - last_auc}")
    epoch += 1
    if abs(delta) < 0.0001:
        print(f"stop at epoch:{epoch}!")
        break


In [47]:
lttttt = label_data.copy()
lttttt.append(
    pd.DataFrame({"label": [1 for i in range(list(pred_prob_A[:,0] > 0.95).count(True))]}),
    ignore_index=True
)

pos_item = data[pred_prob_A[:,0] > 0.9]
pos_item.shape[0]

21813